In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import utils
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [27]:
import keras
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization
from keras.models import load_model
import h5py

In [ ]:
filepath="weights.best.hdf5"


In [3]:
train_data_path = '/home/ec2-user/Capstone_Project/code/left-pca16' # Full path should be given here 
train_mask_path = 'mask/train/'
test_data_path = '/home/ec2-user/Capstone_Project/code/right-pca16' # Full path should be given here 
test_mask_path = 'mask/test/'

In [4]:
# Read data form training and test 

In [15]:
X_train,Y_train, num_bands = utils.load_data(data_path = train_data_path, 
                                             masks_path = train_mask_path, 
                                             crop_size = 33, 
                                             num_classes = 6, 
                                             samples = 50000, 
                                             balance = True)



 balance:  True

 number of samples:  300000 

loading class:  1  - samples:  50000
loading class:  2  - samples:  50000
loading class:  3  - samples:  50000
loading class:  4  - samples:  50000
loading class:  5  - samples:  50000
loading class:  6  - samples:  49997


In [19]:
X_test,Y_test, num_bands = utils.load_data(data_path = test_data_path, 
                                           masks_path = test_mask_path, 
                                           crop_size = 33, 
                                           num_classes = 6, 
                                           samples = 10000, 
                                           balance = True)



 balance:  True

 number of samples:  60000 

loading class:  1  - samples:  10000
loading class:  2  - samples:  10000
loading class:  3  - samples:  10000
loading class:  4  - samples:  10000
loading class:  5  - samples:  10000
loading class:  6  - samples:  10000


In [39]:
# creat generato

In [17]:
print(type(X_train), type(Y_train))
print(X_train.shape, Y_train.shape, num_bands)
print(X_test.shape,  Y_test.shape,  num_bands)

<class 'numpy.ndarray'> <class 'numpy.ndarray'>
(299997, 33, 33, 16) (299997, 6) 16
(300000, 33, 33, 16) (300000, 6) 16


In [23]:
X_train = np.array([X_train[i] for i in range(X_train.shape[0])])

In [20]:
X_test = np.array([X_test[i] for i in range(X_test.shape[0])])

In [21]:
Y_train = np.array([Y_train[i,:] for i in range(Y_train.shape[0])])

In [22]:
Y_test = np.array([Y_test[i,:] for i in range(Y_test.shape[0])])

# Build First Model

In [24]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3, 3), strides=(1, 1), input_shape=(33, 33, 16), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), strides=(1, 1), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), strides=(1, 1), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(MaxPooling2D(pool_size=(2, 2),strides=(2, 2)))

model.add(Flatten())
model.add(Dense(128, kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(Activation('softplus'))
model.add(Dropout(0.5))

model.add(Dense(6, kernel_regularizer=l2(0.01)))
model.add(Activation('softmax')) 

In [25]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

In [26]:
model.fit(X_train, Y_train,
          epochs = 20,
          verbose = 1,
          batch_size= 128,
          validation_data=(X_test, Y_test))

Train on 299997 samples, validate on 60000 samples
Epoch 1/20
299997/299997 [==============================] - 90s 301us/step - loss: 0.5569 - acc: 0.9228 - val_loss: 0.7658 - val_acc: 0.8131
Epoch 2/20
299997/299997 [==============================] - 75s 250us/step - loss: 0.2902 - acc: 0.9451 - val_loss: 0.8543 - val_acc: 0.7690
Epoch 3/20
299997/299997 [==============================] - 75s 249us/step - loss: 0.2533 - acc: 0.9511 - val_loss: 1.3585 - val_acc: 0.7530
Epoch 4/20
299997/299997 [==============================] - 75s 250us/step - loss: 0.2274 - acc: 0.9563 - val_loss: 0.8184 - val_acc: 0.8390
Epoch 5/20
299997/299997 [==============================] - 75s 249us/step - loss: 0.2091 - acc: 0.9596 - val_loss: 0.9584 - val_acc: 0.8082
Epoch 6/20
299997/299997 [==============================] - 75s 251us/step - loss: 0.1978 - acc: 0.9622 - val_loss: 1.2270 - val_acc: 0.7024
Epoch 7/20
299997/299997 [==============================] - 75s 251us/step - loss: 0.1904 - acc: 0.9635

In [28]:
model.save('model_1_1.h5')

In [30]:
model = load_model('model_1_1.h5')

In [33]:
# # this make aure all the layers are trainable
# for layer in model.layers:
#     print(layer.trainable)

In [ ]:
# train again

In [35]:
# model.fit(X_train, Y_train,
#           epochs = 20,
#           verbose = 1,
#           batch_size= 128,
#           validation_data=(X_test, Y_test))